In [ ]:
import numpy as np
import pandas as pd
import time
from datetime import timedelta

In [ ]:
np.random.seed(0)

In [ ]:
def dist_euclidiana(x, y):
    dif_xy = x - y
    pow_dif = np.power(dif_xy,2)
    sum_pow = pow_dif.sum()
    dist = np.sqrt(sum_pow)
    return dist
    
def NN(V,X):
    indices = []
    for v in V:
        distancias = []
        for x in X:
            d = dist_euclidiana(v,x)
            distancias.append(d)
        ix = np.argmin(distancias)
        indices.append(ix)
    return indices

# 5)

## a)

In [ ]:
def acuracia(y_real, y_pred):
    comparacoes = y_real==y_pred
    acc = (sum(comparacoes)/len(comparacoes))*100
    return acc

def amostragem_estratificada(X, prop_classes):
  """
  Realiza a amostragem estratificada com base nas proporções das classes na base de dados

  Params:
    - X: Dataframe do pandas
    - prop_classes: proporções de cada classe
  """
  # calcula o número de amostras por cada classe
  tam_x = round((len(X)-1)/3)
  numero_classes = (prop_classes*tam_x)/100
  
  # separa os numeros e obtem um valor inteiro
  nc1 = round(numero_classes[1])
  nc2 = round(numero_classes[2])
  nc3 = round(numero_classes[3])
  
  # seleciona as bases de cada classe
  classe_1 = X[X[0]==1].index    
  classe_2 = X[X[0]==2].index
  classe_3 = X[X[0]==3].index
  bases = []
  for i in range(3):
    # seleciona indices aleatorios de cada base
    split_base1 = np.random.choice(classe_1,size=nc1, replace=False)
    split_base2 = np.random.choice(classe_2,size=nc2, replace=False)
    split_base3 = np.random.choice(classe_3,size=nc3, replace=False)

    # seleciona as amostras correspondentes
    base1 = X.iloc[split_base1]
    base2 = X.iloc[split_base2]
    base3 = X.iloc[split_base3]
    
    # une as bases de dados
    temp_bases = [base1, base2, base3]
    base = pd.concat(temp_bases)    
    bases.append(base)
  return bases

In [ ]:
def avaliar_atributos(X_treino, X_teste, y_treino, y_teste, atributos):
  """
  Avalia a acuracia de um conjunto de atributos

  Params:
    - X_treino: base de treino
    - X_teste: base de teste
    - y_treino: classes da base treino
    - y_teste: classes da base de teste
    - atributos: lista de atributos
  """
  # seleciona amostras correspondentes aos atributos 
  X_tr = X_treino[:, atributos]
  X_ts = X_teste[:, atributos]
  
  # classifica a base de treino
  vizinhos = NN(X_ts, X_tr)
  y_pred = y_treino[vizinhos]
  # obtem a acurácia
  acc = acuracia(y_teste, y_pred)
  return acc

def SFS(X_treino, X_teste, y_treino, y_teste, natributos=3):
  """
  Executa o algoritmo Sequential Foward Selection

  Params:
    - X_treino: base detreino
    - X_teste: base de teste
    - y_treino: classes da base treino
    - y_teste: classes da base de teste
    - natributos: número de atributos desejados
  """

  atributos = []
  acuracias = []
  
  colunas = np.arange(1,14)
  # encontra o primeiro atributo
  for c in colunas:
      X_tr = X_treino[:,c]
      X_ts = X_teste[:,c]
      vizinhos = NN(X_tr, X_ts)
      y_pred = y_treino[vizinhos]
      acc = acuracia(y_teste,y_pred)
      acuracias.append(acc)
  melhor_atr = np.argmax(acuracias)+1
  # adiciona atributo na lista
  atributos.append(melhor_atr)
  # encontra o restante dos atributos
  for i in range(natributos):
      acuracias = []
      for c in colunas:
          if c not in atributos:
              temp_atr = atributos.copy()                
              temp_atr.append(c)
              X_tr = X_treino[:,temp_atr]
              X_ts = X_teste[:,temp_atr]
              vizinhos = NN(X_tr, X_ts)
              y_pred = y_treino[vizinhos]
              acc = acuracia(y_teste,y_pred)
          else:
              acc = 0
          acuracias.append(acc)
      # adiciono +1 pois removi o atributo 0 do cálculo
      melhor_atr = np.argmax(acuracias)+1
      atributos.append(melhor_atr)
      
  return atributos

def SBE(X_treino, X_teste, y_treino, y_teste, natributos = 3):
  """
  Executa o algoritmo Sequential Bacward Elimination

  Params:
    - X_treino: base detreino
    - X_teste: base de teste
    - y_treino: classes da base treino
    - y_teste: classes da base de teste
    - natributos: número de atributos desejados
  """
  colunas = list(np.arange(1,14))
  
  acuracias = np.zeros(len(colunas))
  atributos = colunas.copy()
  dict_acc = {}
  # obtem o pior atributo
  for i in range(len(colunas)):
      temp_atributos = colunas.copy()
      cr = temp_atributos.pop(i)
      X_tr = X_treino[:,temp_atributos]
      X_ts = X_teste[:,temp_atributos]
      vizinhos = NN(X_tr, X_ts)
      y_pred = y_treino[vizinhos]
      acc = acuracia(y_teste,y_pred)
      acuracias[cr-1] = acc
      dict_acc[cr] = acc
      
      
  pior_atr = np.argmax(acuracias) + 1    
  
  ix_p = atributos.index(pior_atr)
  # remove o pior atributo
  atributos.pop(ix_p)
  
  # remove o pior atributo enquanto o numero de atributos não foi alcançado
  while (len(atributos)>natributos):
      acuracias = np.zeros(len(colunas))        
      for i in range(len(atributos)):
          temp_atributos = atributos.copy()
          cr = temp_atributos.pop(0)
          X_tr = X_treino[:,temp_atributos]
          X_ts = X_teste[:,temp_atributos]
          vizinhos = NN(X_tr, X_ts)
          y_pred = y_treino[vizinhos]
          acc = acuracia(y_teste,y_pred)
          acuracias[cr-1] = acc
      pior_atr = np.argmax(acuracias) + 1
      ix_p = atributos.index(pior_atr)
      atributos.pop(ix_p)
          
  return atributos

In [ ]:
## colunas
# 1) Alcohol (classe) 2) Malic acid 3) Ash 4) Alcalinity of ash 5) Magnesium 6) Total phenols 7) Flavanoids 
# 8)Nonflavanoid phenols 9) Proanthocyanins 10)Color intensity 11)Hue 12)OD280/OD315 of diluted wines 13)Proline

# leitura da base de dados
wine_data = pd.read_csv("wine.data", header=None)

In [ ]:
# proporção das classes
proporcao_classes = round(wine_data[0].value_counts()/(len(wine_data)-1), ndigits=2)*100
proporcao_classes

2    40.0
1    33.0
3    27.0
Name: 0, dtype: float64

In [ ]:
def classificacao_NN(base_treino, base_teste):
  """
  Classficador NN

  Params:
    - base_treino: base de treino
    - base_teste: base de teste
  """
  indices = []
  for x in base_teste.values:
      ix = NN(x, base_treino)
      indices.append(ix)
  pred_labels = base_treino[0].values[indices]
  real_labels = base_teste[0].values
  return pred_labels, real_labels

In [ ]:
treino, teste, validacao = amostragem_estratificada(wine_data,proporcao_classes)
y_treino = treino[0].values
y_teste = teste[0].values
y_validacao = validacao[0].values

# juncao das bases de treino e teste
treino_teste = pd.concat([treino,teste])
y_treinoteste = np.concatenate([y_treino,y_teste])

### SFS

In [ ]:
start_time = time.monotonic()
# escolha dos melhores atributos
melhores_atributos = SFS(treino.values, teste.values, y_treino, y_teste)
# avaliacao dos atributos na terceira base
acc_sfs = avaliar_atributos(treino_teste.values, validacao.values, y_treinoteste, y_validacao, melhores_atributos)
end_time = time.monotonic()
print("Duração", timedelta(seconds=end_time - start_time))
print("Acuracia", acc_sfs)

Duraçao 0:00:03.050909
Acuracia 96.61016949152543


### SBE

In [ ]:
start_time = time.monotonic()
melhores_atributos = SBE(treino.values, teste.values, y_treino, y_teste)
# juncao das bases de treino e teste
treino_teste = pd.concat([treino,teste])
y_treinoteste = np.concatenate([y_treino,y_teste])
# avaliacao dos atributos na terceira base
acc_sbe = avaliar_atributos(treino_teste.values, validacao.values, y_treinoteste, y_validacao, melhores_atributos)
end_time = time.monotonic()
print("Duração", timedelta(seconds=end_time - start_time))
print("Acuracia",acc_sbe)

Duraçao 0:00:05.473326
Acuracia 93.22033898305084


## b)

### SFS

In [ ]:
start_time = time.monotonic()
# escolha dos melhores atributos
melhores_atributos = SFS(treino.values, teste.values, y_treino, y_teste, natributos=8)
# avaliacao dos atributos na terceira base
acc_sfs = avaliar_atributos(treino_teste.values, validacao.values, y_treinoteste, y_validacao, melhores_atributos)
end_time = time.monotonic()
print("Duração",timedelta(seconds=end_time - start_time))
print("Acuracia",acc_sfs)

Duração 0:00:05.049044
Acuracia 88.13559322033898


### SBE

In [ ]:
start_time = time.monotonic()
melhores_atributos = SBE(treino.values, teste.values, y_treino, y_teste,natributos=8)
# avaliacao dos atributos na terceira base
acc_sbe = avaliar_atributos(treino_teste.values, validacao.values, y_treinoteste, y_validacao, melhores_atributos)
end_time = time.monotonic()
print ("Duração", timedelta(seconds=end_time - start_time))
print("Acurácia", acc_sbe)

Duração 0:00:03.701555
Acurácia 86.4406779661017


### c)

**procedimento a)**

In [ ]:
start_time = time.monotonic()
# escolha dos melhores atributos
melhores_atributos = SFS(treino_teste.values, treino_teste.values, y_treinoteste, y_treinoteste)
# avaliacao dos atributos na terceira base
acc_sfs = avaliar_atributos(treino_teste.values, validacao.values, y_treinoteste, y_validacao, melhores_atributos)
end_time = time.monotonic()
print ("Duração", timedelta(seconds=end_time - start_time))
print("Acurácia", acc_sfs)

Duração 0:00:11.433431
Acurácia 96.61016949152543


In [ ]:
start_time = time.monotonic()
melhores_atributos = SBE(treino_teste.values, treino_teste.values, y_treinoteste, y_treinoteste)
# avaliacao dos atributos na terceira base
acc_sbe = avaliar_atributos(treino_teste.values, validacao.values, y_treinoteste, y_validacao, melhores_atributos)
end_time = time.monotonic()
print ("Duração", timedelta(seconds=end_time - start_time))
print("Acurácia", acc_sbe)

Duração 0:00:20.655296
Acurácia 83.05084745762711


**procedimento b)**

In [ ]:
start_time = time.monotonic()
# escolha dos melhores atributos
melhores_atributos = SFS(treino_teste.values, treino_teste.values, y_treinoteste, y_treinoteste,natributos=8)
# avaliacao dos atributos na terceira base
acc_sfs = avaliar_atributos(treino_teste.values, validacao.values, y_treinoteste, y_validacao, melhores_atributos)
end_time = time.monotonic()
print ("Duração", timedelta(seconds=end_time - start_time))
print("Acurácia", acc_sfs)

Duração 0:00:19.953875
Acurácia 86.4406779661017


In [ ]:
start_time = time.monotonic()
melhores_atributos = SBE(treino_teste.values, treino_teste.values, y_treinoteste, y_treinoteste,natributos=8)

# avaliacao dos atributos na terceira base
acc_sbe = avaliar_atributos(treino_teste.values, validacao.values, y_treinoteste, y_validacao, melhores_atributos)
end_time = time.monotonic()
print ("Duração", timedelta(seconds=end_time - start_time))
print("Acurácia", acc_sbe)

Duração 0:00:14.004874
Acurácia 83.05084745762711


A acurácia na terceira parte foi pior do que o resultado obtido nas letras a) e b). Isso ocorre porque utilizar a mesma base como treino e validação, faz com que o modelo classifique com maior acurácia apenas bases praticamente iguais à base utilizada durante o treino, ou seja, o modelo possui pouca generalização.

### d)

**Tabela com tempos de execução (em segundos) dos algoritmos SFS e SBE**

|    |    | SFS   | SBE   |
|----|----|-------|-------|
| a) |    | 3.05  | 5.47  |
| b) |    | 5.04  | 3.70  |
| c) | a) | 11.43 | 20.65 |
|    | b) | 19.95 | 14.00 |

Pela tabela notamos que o SFS é mais rápido para a execução do procedimento a) (seleção de 3 atributos). Além disso, o SBE mostra-se mais rápido que o SFS para a seleção de um número grande de atributos. Em outras palavras, para selecionar um número grande de atributos, o SBE seria o método ideal.